In [35]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import seaborn as sns
import sklearn as sk
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve
from sklearn.metrics import log_loss
from sklearn.metrics import brier_score_loss
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [2]:
games = pd.read_csv('games.csv')

In [3]:
games.columns

Index(['id', 'rated', 'created_at', 'last_move_at', 'turns', 'victory_status',
       'winner', 'increment_code', 'white_id', 'white_rating', 'black_id',
       'black_rating', 'moves', 'opening_eco', 'opening_name', 'opening_ply'],
      dtype='object')

I am interested in determining how the difference between two ratings relates to the probability of winning for each player. 

For example, suppose that we both meet to play a chess match. I, having been a decent player, have a rating of 1800. You just started playing the other day so you have a rating of 1000. The idea behind a rating is to let us know that we are not evently matched. Given my rating, you should know that I will probably crush you, and your chance of winning is dismally lower than mine. So how can I quantify this intuition? 

My intuition tells me that the bigger the difference, the more likely the person with the higher rating will win.

Our ingredients for this cake will be a few new variables:

1) diff_rating will quantify the difference between the player ratings,

2) abs_diff_rating will simply take the absolute value of the quantity above.

3) hr_won will be a binary variable taking 1 if the person with the higher rating won, and 0 if they didn't.

4) higher_rating_coded will determine whether white won (takes value of 1, 0 otherwise).


Having created these variables, we intend to use the abs_diff_rating to predict whether hr_won will be a 1 or a 0 using a logistic regression. 

In [4]:
# Create a new variable for the difference in rating between two players.
games['diff_rating'] = games.white_rating - games.black_rating

In [5]:
# Create a new variable with absolute value of differences
games['abs_diff_rating'] = np.abs(games['diff_rating'])

In [6]:
# Create new key
# -- higher_rating will encode who has the higher rating. 
games['higher_rating'] = ''

In [7]:
# Given the organization of the data, positive differences in the diff_rating indicate white had a higher rating, 
# whereas negative ones indicated a higher rating for black. A difference of zero, of course, indicates an equal rating. 
games.loc[games.diff_rating > 0, 'higher_rating'] = 'white'
games.loc[games.diff_rating < 0, 'higher_rating'] = 'black'
games.loc[games.diff_rating == 0, 'higher_rating'] = 'same'

In [12]:
# Now, we encode a variable that returns 1 if the higher rating won and 0 otherwise. 
games['hr_won'] = 0
games.loc[games.winner == games.higher_rating, 'hr_won'] = 1

In [15]:
# A cursory check shows this worked.
games[['hr_won', 'winner', 'higher_rating']].head()

,hr_won,winner,higher_rating
0,1,white,white
1,0,black,white
2,0,white,black
3,0,white,black
4,1,white,white


In [16]:
# Next, we will code higher rating as a binary so that we can easily use it 
games['higher_rating_coded'] = 0
games.loc[games.higher_rating == 'white', 'higher_rating_coded'] = 1

Now would be a good time to show the variables we just created. Also, let's see the original winner variable. 

In [34]:
# Randomly select a sample from the variables we created. 
# We want to see if our created variables match up
rand_ind = np.random.randint(len(games), size = 10)
games.iloc[rand_ind][['higher_rating','higher_rating_coded', 'diff_rating', 'abs_diff_rating', 'hr_won', 'winner']]

,higher_rating,higher_rating_coded,diff_rating,abs_diff_rating,hr_won,winner
5734,black,0,-66,66,1,black
6812,black,0,-462,462,0,white
8720,white,1,40,40,1,white
505,white,1,147,147,0,black
13054,white,1,34,34,1,white
2241,black,0,-649,649,1,black
16027,black,0,-63,63,1,black
5912,white,1,95,95,1,white
10426,same,0,0,0,0,black
11930,black,0,-229,229,0,white


With this small output, we can see that we've done what we set out to do.

In [41]:
# Output a new dataset with the variables we just created. 
games.to_csv('games_new_vars.csv')

Now, we will take abs_diff_rating and use it as our predictor for hr_won using a logistic regression.

In [42]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5)

In [47]:
X_test.shape

(10029, 1)

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [38]:
# Divide our data into the response and explanatory variable
X = games[['abs_diff_rating']]
Y = games.hr_won

We now need to split our data into a training and test data set. We will first try using k-fold cross validation as a way of estimating our test MSE. 

In [39]:
# Use five folds
kf =KFold( n_splits = 5)
for train, test in kf.split(X, Y):
    print(train)
    print(test)

[ 4012  4013  4014 ... 20055 20056 20057]
[   0    1    2 ... 4009 4010 4011]
[    0     1     2 ... 20055 20056 20057]
[4012 4013 4014 ... 8021 8022 8023]
[    0     1     2 ... 20055 20056 20057]
[ 8024  8025  8026 ... 12033 12034 12035]
[    0     1     2 ... 20055 20056 20057]
[12036 12037 12038 ... 16044 16045 16046]
[    0     1     2 ... 16044 16045 16046]
[16047 16048 16049 ... 20055 20056 20057]


In [11]:
# Can I add a variable to make this a better model?
games.columns

Index(['id', 'rated', 'created_at', 'last_move_at', 'turns', 'victory_status',
       'winner', 'increment_code', 'white_id', 'white_rating', 'black_id',
       'black_rating', 'moves', 'opening_eco', 'opening_name', 'opening_ply',
       'diff_rating', 'abs_diff_rating', 'higher_rating', 'hr_won'],
      dtype='object')

I'm going to try and add two more variables here. 

1) White is known to have an advantage (how subtle, you can look up here:) over black by getting to move first. So, I intend to code a white_higher_rated
(white higher rating) binary variable. 1 if White had the higher rating 0 otherwise. This may help show how being white or black affects having a higher rating over the other player. 

2) Number of turns. I hypothesize that longer turned games usually result in more venly matched games, despite the rating. Perhaps by knowing the number of turns made, I can say something about the probability of the higher rated player winning. 

-> Come in Bayes? Information.

In [12]:
# Code higher rated player, 1 they are white, 0 ow
games['white_higher_rated'] = 0
for i in list(range(len(games))):
    if games.higher_rating[i] == 'white':
        games.white_higher_rated[i] = 1

/Users/Leonard/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [13]:
# Check values
games[['white_higher_rated', 'higher_rating']].tail()

,white_higher_rated,higher_rating
20053,1,white
20054,1,white
20055,0,black
20056,1,white
20057,0,black


In [23]:
# Code up the variables to use
X = games[['abs_diff_rating', 'turns', 'white_higher_rated']]
Y = games.hr_won

In [18]:
# Split into test and training data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=0)

In [ ]:
sum(mult_lg.predict(X_test) - Y_test) / len(Y_test)

In [ ]:
mult_lg.score(X_test, Y_test)

In [ ]:
probs = mult_lg.predict_proba(X_train)

In [ ]:
print('logistic loss = ', log_loss(y_true=Y_test, y_pred=probs))

Uh oh. Probably won't be keep you around...

In [ ]:

# After figuring out how the formula works, the below could highlight the difference better. 

In [ ]:
var = 500

In [ ]:
np.e ** (clf.intercept_[0] + var * clf.coef_[0][0]) / ( 1 + np.e ** (clf.intercept_[0] + var * clf.coef_[0][0]) )